In [86]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import re
import joblib
from numpy.random import seed
from numpy.random import randn
import torch
from ANN_model_transfer_learning import prediction_model

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

from scipy.stats import mannwhitneyu

# regression_model_prediction

In [87]:
ANN_model_transfer_learning = torch.load('./model/ANN_model_transfer_learning.pth')
ANN_model_transfer_learning.eval()
# ANN_model

prediction_model(
  (hidden1): Linear(in_features=1024, out_features=512, bias=True)
  (batch1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden2): Linear(in_features=512, out_features=512, bias=True)
  (dropout2): Dropout(p=0.01, inplace=False)
  (batch2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predict): Linear(in_features=512, out_features=1, bias=True)
)

## fingerprint calc.

In [88]:
def smiles_dataset(dataset_df = None, smiles_loc = 'Smiles', fp_radius = 3, fp_bits = 1024):

    '''
    Use this function to generate the dataframe of fingerprint
    dataset_df: the input dataset should be a dataframe
    smiles_loc: the column name that consists of SMILES strings
    fp_radius = the radius of Morgan fingerprint
    fp_bits = the number of fingerprint bits of Morgan fingerprint
    '''

    smiles = dataset_df[smiles_loc]
    smiles_list = np.array(smiles).tolist()

    mols = [Chem.MolFromSmiles(smile) for smile in smiles_list]

    mols = [Chem.AddHs(smile) for smile in mols]  # Note that add Hs would change the fingerprint albeit the same molecule

    morgans = [AllChem.GetMorganFingerprintAsBitVect(mol, radius = fp_radius,
                nBits= fp_bits, useChirality = True) for mol in mols]

    morgan_bits =  [morgan.ToBitString() for morgan in morgans]

    pattern = re.compile('.{1}')  # find every single digit
    morgan_bits = [','.join(pattern.findall(morgan)) for morgan in morgan_bits]

    fp_list = []
    for bit in morgan_bits:
        single_fp = bit.split(',')   # split the string by commas
        single_fp = [float(fp) for fp in single_fp] # transfer string to float32
        fp_list.append(single_fp)

    fp_df = pd.DataFrame(np.array(fp_list))
    fp_df.columns = fp_df.columns.astype(str)

    # rename the columns
    for i in range(fp_df.columns.shape[0]):
        fp_df.rename(columns = {fp_df.columns[i]:fp_df.columns[i] + "pIC50"}, inplace = True)

    return fp_df

## ZINC dataset preparation

In [89]:
ZINC_data = pd.read_csv('./ZINC Data/CAAD.txt', sep='\t')
ZINC_data = ZINC_data[['smiles', 'zinc_id']]
ZINC_data = ZINC_data.dropna(subset=['zinc_id']).reset_index(drop=True)

# ZINC_data[ZINC_data['smiles' == 'CC(']]

In [90]:
x_ZINC = smiles_dataset(dataset_df = ZINC_data, smiles_loc = 'smiles')
x_ZINC_copy = x_ZINC.copy()

## prediction

In [91]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
x_ZINC = x_ZINC.values
x_ZINC = torch.tensor(x_ZINC, dtype = torch.float32).to(device)

y_pred = ANN_model_transfer_learning(x_ZINC)
y_pred

cuda


tensor([[4.7574],
        [4.8402],
        [5.3481],
        ...,
        [5.6664],
        [4.8120],
        [6.1375]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [92]:
pred_list = [i.detach().cpu().numpy()[0] for i in y_pred]
pred = pd.Series(pred_list, name='pred_pIC50')
pred

0        4.757380
1        4.840161
2        5.348131
3        5.903056
4        5.659246
           ...   
48931    5.122999
48932    4.963513
48933    5.666362
48934    4.811950
48935    6.137493
Name: pred_pIC50, Length: 48936, dtype: float32

In [93]:
ZINC_with_pred_pIC50 = pd.concat([ZINC_data, pred], axis = 1)
ZINC_with_pred_pIC50

,smiles,zinc_id,pred_pIC50
0,Cc1nn(C)c(C)c1S(=O)(=O)N[C@@H](CO)C(=O)O,3.696775e+07,4.757380
1,Cn1cc(CNC(=O)CCNS(C)(=O)=O)cn1,4.909435e+07,4.840161
2,O=C(O)[C@H]1CN(C(=O)[C@@H]2CCS(=O)(=O)C2)CCO1,4.986487e+07,5.348131
3,CS(=O)(=O)CCS(=O)(=O)NCc1cc[nH]n1,6.314354e+07,5.903056
4,NC(=O)Cn1ccc(NC(=O)[C@@H]2CCC(=O)N2)n1,6.820913e+07,5.659246
...,...,...,...
48931,Cn1cncc1CCC(=O)N[C@@H]1CS(=O)(=O)C[C@H]1O,1.412406e+09,5.122999
48932,CCOC(=O)CN(C)C(=O)Cn1nnc(C(N)=O)n1,1.412931e+09,4.963513
48933,O=C(NCCO)C1CCN(C(=O)CN2CCCC2=O)CC1,1.413352e+09,5.666362
48934,CC1=C(C(=O)N[C@@H]2C[C@H](O)[C@H](O)C2)Cn2nnnc...,1.413628e+09,4.811950


## regression classification

In [94]:
bioactivity_threshold = []
for i in ZINC_with_pred_pIC50['pred_pIC50']:
  if float(i) >= 6.0:
    bioactivity_threshold.append('active')  # inactive == 0
  elif float(i) < 6.0:
    bioactivity_threshold.append('inactive')    # active == 1
  
  
ZINC_with_pred_pIC50['class_by_regression'] = bioactivity_threshold
ZINC_with_pred_pIC50_class = ZINC_with_pred_pIC50
ZINC_with_pred_pIC50_class

,smiles,zinc_id,pred_pIC50,class_by_regression
0,Cc1nn(C)c(C)c1S(=O)(=O)N[C@@H](CO)C(=O)O,3.696775e+07,4.757380,inactive
1,Cn1cc(CNC(=O)CCNS(C)(=O)=O)cn1,4.909435e+07,4.840161,inactive
2,O=C(O)[C@H]1CN(C(=O)[C@@H]2CCS(=O)(=O)C2)CCO1,4.986487e+07,5.348131,inactive
3,CS(=O)(=O)CCS(=O)(=O)NCc1cc[nH]n1,6.314354e+07,5.903056,inactive
4,NC(=O)Cn1ccc(NC(=O)[C@@H]2CCC(=O)N2)n1,6.820913e+07,5.659246,inactive
...,...,...,...,...
48931,Cn1cncc1CCC(=O)N[C@@H]1CS(=O)(=O)C[C@H]1O,1.412406e+09,5.122999,inactive
48932,CCOC(=O)CN(C)C(=O)Cn1nnc(C(N)=O)n1,1.412931e+09,4.963513,inactive
48933,O=C(NCCO)C1CCN(C(=O)CN2CCCC2=O)CC1,1.413352e+09,5.666362,inactive
48934,CC1=C(C(=O)N[C@@H]2C[C@H](O)[C@H](O)C2)Cn2nnnc...,1.413628e+09,4.811950,inactive


In [95]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [96]:
df_lipinski_ZINC= lipinski(ZINC_with_pred_pIC50_class['smiles'])
df_lipinski_ZINC

,MW,LogP,NumHDonors,NumHAcceptors
0,277.302,-1.23926,3.0,6.0
1,260.319,-1.02440,2.0,5.0
2,277.298,-1.26690,1.0,5.0
3,267.332,-1.12630,2.0,5.0
4,251.246,-1.41450,3.0,5.0
...,...,...,...,...
48931,287.341,-1.37320,2.0,6.0
48932,270.249,-2.20650,1.0,8.0
48933,297.355,-1.04410,2.0,4.0
48934,294.315,-1.60270,3.0,8.0


In [97]:
complete_ZINC = pd.concat([ZINC_with_pred_pIC50_class, df_lipinski_ZINC], axis=1)
complete_ZINC

,smiles,zinc_id,pred_pIC50,class_by_regression,MW,LogP,NumHDonors,NumHAcceptors
0,Cc1nn(C)c(C)c1S(=O)(=O)N[C@@H](CO)C(=O)O,3.696775e+07,4.757380,inactive,277.302,-1.23926,3.0,6.0
1,Cn1cc(CNC(=O)CCNS(C)(=O)=O)cn1,4.909435e+07,4.840161,inactive,260.319,-1.02440,2.0,5.0
2,O=C(O)[C@H]1CN(C(=O)[C@@H]2CCS(=O)(=O)C2)CCO1,4.986487e+07,5.348131,inactive,277.298,-1.26690,1.0,5.0
3,CS(=O)(=O)CCS(=O)(=O)NCc1cc[nH]n1,6.314354e+07,5.903056,inactive,267.332,-1.12630,2.0,5.0
4,NC(=O)Cn1ccc(NC(=O)[C@@H]2CCC(=O)N2)n1,6.820913e+07,5.659246,inactive,251.246,-1.41450,3.0,5.0
...,...,...,...,...,...,...,...,...
48931,Cn1cncc1CCC(=O)N[C@@H]1CS(=O)(=O)C[C@H]1O,1.412406e+09,5.122999,inactive,287.341,-1.37320,2.0,6.0
48932,CCOC(=O)CN(C)C(=O)Cn1nnc(C(N)=O)n1,1.412931e+09,4.963513,inactive,270.249,-2.20650,1.0,8.0
48933,O=C(NCCO)C1CCN(C(=O)CN2CCCC2=O)CC1,1.413352e+09,5.666362,inactive,297.355,-1.04410,2.0,4.0
48934,CC1=C(C(=O)N[C@@H]2C[C@H](O)[C@H](O)C2)Cn2nnnc...,1.413628e+09,4.811950,inactive,294.315,-1.60270,3.0,8.0


## p-value

In [98]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'class_by_regression']
  df = complete_ZINC[selection]
  active = df[df['class_by_regression'] == 'active']
  active = active[descriptor]

  selection = [descriptor, 'class_by_regression']
  df = complete_ZINC[selection]
  inactive = df[df['class_by_regression'] == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'
  
  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [99]:
mannwhitney('pred_pIC50')

,Descriptor,Statistics,p,alpha,Interpretation
0,pred_pIC50,275582399.0,0.0,0.05,Different distribution (reject H0)


In [100]:
mannwhitney('MW')

,Descriptor,Statistics,p,alpha,Interpretation
0,MW,139223609.5,0.176635,0.05,Same distribution (fail to reject H0)


In [101]:
mannwhitney('LogP')

,Descriptor,Statistics,p,alpha,Interpretation
0,LogP,141192681.5,0.001334,0.05,Different distribution (reject H0)


In [102]:
mannwhitney('NumHDonors')

,Descriptor,Statistics,p,alpha,Interpretation
0,NumHDonors,176421393.0,0.0,0.05,Different distribution (reject H0)


In [103]:
mannwhitney('NumHAcceptors')

,Descriptor,Statistics,p,alpha,Interpretation
0,NumHAcceptors,145013527.5,1.625418e-12,0.05,Different distribution (reject H0)


# classification_model_prediction

In [104]:
x_classification = pd.concat([x_ZINC_copy, df_lipinski_ZINC[['LogP', 'MW']]], axis=1)
x_classification

,0pIC50,1pIC50,2pIC50,3pIC50,4pIC50,5pIC50,6pIC50,7pIC50,8pIC50,9pIC50,...,1016pIC50,1017pIC50,1018pIC50,1019pIC50,1020pIC50,1021pIC50,1022pIC50,1023pIC50,LogP,MW
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.23926,277.302
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.02440,260.319
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.26690,277.298
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.12630,267.332
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-1.41450,251.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.37320,287.341
48932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.20650,270.249
48933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.04410,297.355
48934,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.60270,294.315


In [105]:
LogisticRegression = torch.load('./model/LogisticRegression.pth')
LogisticRegression

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=64)

In [106]:
LogisticRegression_result = LogisticRegression.predict(x_classification)
LogisticRegression_result = pd.Series(LogisticRegression_result, name = 'classfication_result')

In [107]:
result = pd.concat([ZINC_with_pred_pIC50_class, LogisticRegression_result], axis = 1)
result

,smiles,zinc_id,pred_pIC50,class_by_regression,classfication_result
0,Cc1nn(C)c(C)c1S(=O)(=O)N[C@@H](CO)C(=O)O,3.696775e+07,4.757380,inactive,inactive
1,Cn1cc(CNC(=O)CCNS(C)(=O)=O)cn1,4.909435e+07,4.840161,inactive,inactive
2,O=C(O)[C@H]1CN(C(=O)[C@@H]2CCS(=O)(=O)C2)CCO1,4.986487e+07,5.348131,inactive,inactive
3,CS(=O)(=O)CCS(=O)(=O)NCc1cc[nH]n1,6.314354e+07,5.903056,inactive,inactive
4,NC(=O)Cn1ccc(NC(=O)[C@@H]2CCC(=O)N2)n1,6.820913e+07,5.659246,inactive,inactive
...,...,...,...,...,...
48931,Cn1cncc1CCC(=O)N[C@@H]1CS(=O)(=O)C[C@H]1O,1.412406e+09,5.122999,inactive,inactive
48932,CCOC(=O)CN(C)C(=O)Cn1nnc(C(N)=O)n1,1.412931e+09,4.963513,inactive,inactive
48933,O=C(NCCO)C1CCN(C(=O)CN2CCCC2=O)CC1,1.413352e+09,5.666362,inactive,inactive
48934,CC1=C(C(=O)N[C@@H]2C[C@H](O)[C@H](O)C2)Cn2nnnc...,1.413628e+09,4.811950,inactive,inactive


In [108]:
same_prediction = result[result['class_by_regression'] == result['classfication_result']]

In [109]:
same_prediction[same_prediction['class_by_regression'] == 'active']

,smiles,zinc_id,pred_pIC50,class_by_regression,classfication_result
20,CN(C)C(=O)CNC(=O)NCCNC(=O)c1ccn[nH]1,2.724802e+08,6.242990,active,active
35,O=C1CCC(=O)N1CCCN1C(=O)N[C@H](CO)C1=O,3.490794e+08,6.736444,active,active
40,COC[C@H](C(N)=O)N(C)C(=O)[C@@H]1CC[C@@H](C(=O)...,3.546104e+08,6.741653,active,active
65,C[C@H](NS(C)(=O)=O)C(=O)N[C@@H]1CC(=O)N(C)C1=O,5.555587e+08,6.145336,active,active
66,Cc1ccc(S(=O)(=O)NCC(=O)NCC(=O)O)cn1,5.604380e+08,6.940023,active,active
...,...,...,...,...,...
48912,COC[C@@H](O)CN[C@H]1C[C@@H](CNC(=O)[C@@H]2CCC(...,1.397614e+09,6.452569,active,active
48913,COCCC(=O)N[C@@H](CNC(=O)Cn1cnnn1)C(C)C,1.399519e+09,6.287286,active,active
48914,CCc1n[nH]cc1C(=O)NC[C@H](O)CNC(=O)COC,1.399765e+09,6.295420,active,active
48919,CC(=O)N1CC(C(=O)NC[C@H](C)CNCC(=O)N(C)C)C1,1.401105e+09,6.575352,active,active
